In [12]:
'''import torch
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from torchtext.vocab import GloVe
from collections import Counter
import pandas as pd
import subprocess as sp'''

import subprocess as sp
import pandas as pd
import os

In [13]:
from src import embed

In [14]:
base_path = sp.getoutput('git rev-parse --show-toplevel')
data_dir = 'data'
file = 'utterances_clean2014-2018.csv'
# csv loaded up from 'text_cleaning/utterances_flattener.ipynb'
df = pd.read_csv(os.path.join(base_path, data_dir, file))

### Testing GloVe

In [5]:
glove_test = embed.get_glove_embeddings(df['text'], 100)

/Users/michaelwagner/Dropbox/Q3/advanced_ml/supreme_court_nlp/src/embed.py:39: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  embeddings = [torch.tensor(sentence) for sentence in embeddings_list]


In [6]:
# 73531 text entries
# sentences padded to 1915 (longest sentence)
# each word dimension 100
glove_test.shape

torch.Size([73531, 1915, 100])

In [7]:
# one sentence
glove_test[0]

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2252,  0.5219,  0.4974,  ...,  0.0550,  0.4561,  0.6579],
        [-0.0378,  0.5388,  0.7541,  ...,  0.1866,  0.4622,  0.3770],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

### Testing BoW

In [8]:
bow_test = embed.get_bow_embeddings(df['text'], 5000)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1330: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [9]:
# 73531 text
# 5000 width for vocab
bow_test.shape

torch.Size([73531, 5000])

In [10]:
# one sentence
bow_test[0]

tensor([0, 0, 0,  ..., 0, 0, 0])

In [11]:
# look at most frequent words in a text value
bow_test[9].topk(10)

torch.return_types.topk(
values=tensor([8, 6, 5, 4, 3, 2, 2, 2, 2, 2]),
indices=tensor([4589, 4590, 2719, 2887, 3348, 4143, 1150, 1023, 4221, 2880]))